# Test forward_projector.py
## Test to generate peaks forward computed from an experimentally reconstructed grain map and convert peaks to cf
## Haixing Fang
## Jan 2025

In [ ]:
import os

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# import functions we need

import concurrent.futures

# %matplotlib ipympl

import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
import ImageD11.parameters
import ImageD11.unitcell
import time
from joblib import Parallel, delayed

In [ ]:
from ImageD11.forward_model import forward_projector
from ImageD11.forward_model import io
from ImageD11.forward_model import forward_model
from ImageD11.forward_model import pars_conversion
from ImageD11.forward_model import grainmaps

In [ ]:
import logging
from tqdm import tqdm
import numba

In [ ]:
# sample_filename can be xx_grains.h5, pbp_tensormap_refined.h5, in which the regional voxels would be merged to generate 'labels'
# DS.h5 stores the grain map after merging regions and identifying grain IDs
# sample_filename = 'A2050_DN_W340_nscope_full_slice_grains.h5'
# sample_filename = 'pbp_tensormap_refined.h5'
sample_filename = 'DS.h5'
pars_filename = '/data/visitor/ma6288/id11/20241119/PROCESSED_DATA/nscope_pars/pars.json'
phase_name = 'Al'

In [ ]:
opts = {
        "energy": 43.56,                 # [keV]
        "beam_size": [1e-3, 1e-3],       # [mm]
        "beam_profile": "gaussian",      # [-]
        "flux": 5e14,                    # [photons/s]
        "Lss": 0.0,                      # [mm]
        "min_misori": 3.0,               # [deg]
        "crystal_system": 'cubic',
        "remove_small_grains": True,
        "min_vol": 3,                    # [voxel]
        "rou": 2.7,                      # [g/cm^3]
        "mass_abs": 0.56685,             # [cm^2/g]
        "y0_offset": 0.0,                # [um]
        "exp_time": 0.002,               # [s]
        "rot_start": -89.975,            # [deg]
        "rot_end": 90.9668,              # [deg]
        "rot_step": 0.05,                # [deg]
        "sparse_omega": True,
        "halfy": 182.0,                  # [um]
        "dty_step": 1.0,                 # [um]
        "ds_max": 1.2,                   # [1/angstrom]
        "plot_peaks": False,
        "plot_flag": False,
        "detector": "eiger",
        "int_factors": (0.1065, 0.7807, 0.1065),
        "slurm_folder": "slurm_fwd_proj_Al"}

In [ ]:
fp = forward_projector.forward_projector(sample_filename, pars_filename, phase_name, detector_mask = None, to_sparse = False, **opts)

# Run all dty calculations on cluster

In [ ]:
# No need to produce sparse files
fp.to_sparse

In [ ]:
# send jobs to the cluster
fp.args['use_cluster']

In [ ]:
# run jobs for all dty positions, if the output peaks file is already existed, it would skip
t0 = time.time()
fp.run_all()
t1 = time.time()
print('It takes {} s'.format(t1 - t0))

In [ ]:
# the peaks are stored in two types:
# 1) cf_2d, cf_3d, cf_4d like cf object in ImageD11
# 2) peaks_2d, peaks_3d, peaks_4d in numpy array with a shape of N*25
fp.read_cf()

In [ ]:
# merge 3D peaks to 4D peaks
fp.get_cf_4d()

In [ ]:
fp.write_cf()

In [ ]:
forward_projector.plot_fwd_peaks(fp.peaks_3d)

In [ ]:
forward_projector.plot_fwd_peaks(fp.peaks_4d)

In [ ]:
# plot sino for 2D peaks
fp.plot_cf(cf_type = '2d', m = None)

In [ ]:
forward_model.cf_plot_sino(fp.cf_3d)

In [ ]:
forward_model.cf_plot_sino(fp.cf_4d)

# Plot 2D peaks from a specific grainID or a specific range of ds

In [ ]:
# specific grainID
fp.plot_cf(cf_type = '2d', m = fp.cf_2d.grainID == 3)

In [ ]:
# specific grainID
fp.plot_cf(cf_type = '2d', m = fp.cf_2d.grainID == 7)

In [ ]:
# specific ds range
fp.plot_cf(cf_type = '2d', m = (fp.cf_2d.ds > 0.4) & (fp.cf_2d.ds < 0.5))

# Compare with experimental peaks

In [ ]:
dset_file = '/data/visitor/ma6288/id11/20241119/PROCESSED_DATA/A2050_DN_W340_nscope_5pct_strained/A2050_DN_W340_nscope_5pct_strained_full_slice_tomo/A2050_DN_W340_nscope_5pct_strained_full_slice_dataset.h5'

ds = ImageD11.sinograms.dataset.load(dset_file)

In [ ]:
# Import 4D peaks

cf_4d = ds.get_cf_4d_from_disk()

ds.update_colfile_pars(cf_4d, phase_name='Al')

print(f"Read {cf_4d.nrows} 4D peaks")

In [ ]:
cf_2d = ds.get_cf_2d()
ds.update_colfile_pars( cf_2d )

In [ ]:
plt.figure()
plt.hist2d( cf_2d.ds, cf_2d.eta, weights= np.log(cf_2d.sum_intensity), bins = (2000, 360), norm='log')
# plt.plot( dssa, np.zeros_like(dssa), '|', ms = 20, lw=1);
plt.show()

In [ ]:
# make a copy of fp.cf_2d
cf_2d_fwd = fp.cf_2d.copy()

In [ ]:
f, a = plt.subplots(1,2,figsize=(14,6), sharex=True, sharey=True)

# simu
m200 = (cf_2d_fwd.ds < 0.5)&(cf_2d_fwd.ds > 0.4)
h0 = a[0].hist2d( cf_2d_fwd.omega[m200], cf_2d_fwd.dty[m200], weights= np.log(cf_2d_fwd.sum_intensity[m200]),
           bins = (fp.omega_angles.shape[0], fp.dtys.shape[0]), norm='log')
# plt.colorbar(label="Log Intensity")
a[0].set_xlabel("Omega ($^{o}$)")
a[0].set_ylabel("dty ($\mu$m)")
a[0].set_title('(a) Simu, {002}')

cbar0 = f.colorbar(h0[3], ax=a[0])
# cbar0.set_label("Log Intensity")

# exp
m200 = (cf_2d.ds < 0.5)&(cf_2d.ds > 0.49)
h1 = a[1].hist2d( cf_2d.omega[m200], cf_2d.dty[ m200], weights=cf_2d.sum_intensity[m200], bins=(ds.obinedges, ds.ybinedges), norm='log')
a[1].set_xlabel("Omega ($^{o}$)")
a[1].set_ylabel("dty ($\mu$m)")
a[1].set_title('(b) Exp, {002}')

cbar1 = f.colorbar(h1[3], ax=a[1])
# cbar1.set_label("Log Intensity")

plt.tight_layout()
plt.show()

In [ ]:
f, a = plt.subplots(1,2,figsize=(14,6), sharex=True, sharey=True)

# simu
m311 = (cf_2d_fwd.ds < 0.83)&(cf_2d_fwd.ds > 0.76)
h0 = a[0].hist2d( cf_2d_fwd.omega[m311], cf_2d_fwd.dty[m311], weights= np.log(cf_2d_fwd.sum_intensity[m311]),
           bins = (fp.omega_angles.shape[0], fp.dtys.shape[0]), norm='log')
cbar0 = f.colorbar(h0[3], ax=a[0])
a[0].set_xlabel("Omega ($^{o}$)")
a[0].set_ylabel("dty ($\mu$m)")
a[0].set_title('(a) Simu, {113}')


# exp
m311 = (cf_2d.ds < 0.83)&(cf_2d.ds > 0.81)
h1 = a[1].hist2d( cf_2d.omega[m311], cf_2d.dty[ m311], weights=cf_2d.sum_intensity[m311], bins=(ds.obinedges, ds.ybinedges), norm='log')
cbar1 = f.colorbar(h1[3], ax=a[1])
a[1].set_xlabel("Omega ($^{o}$)")
a[1].set_ylabel("dty ($\mu$m)")
a[1].set_title('(b) Exp, {113}')
plt.tight_layout()
plt.show()

In [ ]:
m311 = (cf_2d.ds < 0.83)&(cf_2d.ds > 0.81)
# color as dstar
sI = np.histogram2d( cf_2d.omega[m311], cf_2d.dty[ m311], weights=cf_2d.sum_intensity[m311], 
                bins = (ds.obinedges, ds.ybinedges))
sId = np.histogram2d( cf_2d.omega[m311], cf_2d.dty[ m311], weights=cf_2d.sum_intensity[m311] * cf_2d.ds[m311],
                bins = (ds.obinedges, ds.ybinedges))
ds200 = (sId[0] / sI[0]).T
x0 = 0.82
strain = (ds200 - x0)/x0

plt.figure()
plt.pcolormesh( ds.obinedges, ds.ybinedges, strain , cmap = 'RdBu', vmin=-3e-3, vmax=3e-3);
plt.colorbar()
plt.show()

In [ ]:
# plot 4D peaks, simu and exp
forward_model.cf_plot_sino([fp.cf_4d, cf_4d])